# Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
file_to_load = "Resources/purchase_data.csv"

# Reading Purchasing File and storing into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
purchase_data.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [3]:
purchase_data.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


## Player Count

* Display the total number of players


In [4]:
# Getting unique player count from column SN in given data
player_count = purchase_data["SN"].nunique()
player_count_df = pd.DataFrame({"Total Players":[player_count]})
player_count_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
# Getting Unique items count from Item Name column
unique_item = purchase_data["Item Name"].nunique()

# Getting Average Price for complete dataset
average_price = "${:,.2f}".format(purchase_data["Price"].mean())

# Gettting total number of Unique purchases from column Purchase ID
no_purchase = purchase_data["Purchase ID"].count()

# Getting total revenue from column Price
total_price = "${:,.2f}".format(purchase_data["Price"].sum())

# Storing and Displyaing all calculated Metrics using DataFrame
purchase_df = pd.DataFrame({"Number of Unique Item":[unique_item],"Average Price":[average_price],"Number Of Purchase":[no_purchase],"Total Revenue":[total_price]})
purchase_df

,Number of Unique Item,Average Price,Number Of Purchase,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
# Calculating total number of players as per gender
gender_demo_df_ct = purchase_data.groupby(["Gender"]).agg({"SN" : "nunique"})

# Renaming calculated column to read Total Count of Players
gender_demo_df_ct = gender_demo_df_ct.rename(columns={"SN":"Total Count"})

# Using Dataframe created above, calculating % for each gender category and storing in a new Dataframe
gender_demo_df_pc = gender_demo_df_ct.apply(lambda x: 100 * x / float(x.sum()))

# Renaming percentage column to read Percentage of Players
gender_demo_df_pc = gender_demo_df_pc.rename(columns={"Total Count":"Percentage of Players"})

# Combining above 2 Dataframes to get Total Count and Percentage in a single Dataframe
gender_demo_df = pd.concat([gender_demo_df_ct,gender_demo_df_pc], axis=1)

# Sorting the data to display highest count first
gender_demo_df = gender_demo_df.sort_values(by="Total Count", ascending=False)

# Changing display format of the Percentage column
# pd.options.display.float_format = "{:,.2f}%".format # another option, but it will format all Float Value in the program to %
gender_demo_df["Percentage of Players"] = gender_demo_df["Percentage of Players"].map("{:,.2f}%".format)

gender_demo_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


    
## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Calculating Purchase Count
purchase_count = purchase_data.groupby(["Gender"]).agg({"Purchase ID":"count"})
purchase_count = purchase_count.rename(columns={"Purchase ID":"Purchase Count"})

# Calculating Average Purchase Price
avg_purchase = purchase_data.groupby(["Gender"]).agg({"Price":"mean"})
avg_purchase = avg_purchase.rename(columns={"Price":"Average Purchase Price"})
avg_purchase["Average Purchase Price"] = avg_purchase["Average Purchase Price"].map("${:,.2f}".format)

# Calculating Total Purchase value
total_purchase = purchase_data.groupby(["Gender"]).agg({"Price":"sum"})
total_purchase = total_purchase.rename(columns={"Price":"Total Purchase Value"})

# Average Total Purchase per person
avg_total_purchase_df = pd.concat([gender_demo_df,total_purchase], axis=1)
avg_total_purchase_df["Avg Total Purchase Per Person"] = avg_total_purchase_df["Total Purchase Value"]/avg_total_purchase_df["Total Count"]
avg_total_purchase = avg_total_purchase_df["Avg Total Purchase Per Person"].map("${:,.2f}".format)

total_purchase["Total Purchase Value"] = total_purchase["Total Purchase Value"].map("${:,.2f}".format)

# Combining all calculated metrics in single dataframe
purchase_analysis_df = pd.concat([purchase_count,avg_purchase,total_purchase,avg_total_purchase], axis=1)
purchase_analysis_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [34]:
# Calculate player count by Age Group
#player_count = purchase_data.groupby(pd.cut(purchase_data["Age"], np.arange(0, 40, 5))).nunique()["SN"].to_frame(name="Total Count")
player_count_df = pd.DataFrame({"Age Ranges" : ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
                        "Total Count" : [0,0,0,0,0,0,0,0]
})

unique_df = purchase_data.drop_duplicates("SN")

for i in unique_df.index:
    if (purchase_data["Age"][i] < 10):
        player_count_df.loc[0,"Total Count"] = player_count_df.loc[0,"Total Count"] + 1
    elif (purchase_data["Age"][i] >= 10 and purchase_data["Age"][i] <= 14):
        player_count_df.loc[1,"Total Count"] = player_count_df.loc[1,"Total Count"] + 1
    elif (purchase_data["Age"][i] >= 15 and purchase_data["Age"][i] <= 19):
        player_count_df.loc[2,"Total Count"] = player_count_df.loc[2,"Total Count"] + 1
    elif (purchase_data["Age"][i] >= 20 and purchase_data["Age"][i] <= 24):
        player_count_df.loc[3,"Total Count"] = player_count_df.loc[3,"Total Count"] + 1
    elif (purchase_data["Age"][i] >= 25 and purchase_data["Age"][i] <= 29):
        player_count_df.loc[4,"Total Count"] = player_count_df.loc[4,"Total Count"] + 1
    elif (purchase_data["Age"][i] >= 30 and purchase_data["Age"][i] <= 34):
        player_count_df.loc[5,"Total Count"] = player_count_df.loc[5,"Total Count"] + 1
    elif (purchase_data["Age"][i] >= 35 and purchase_data["Age"][i] <= 39):
        player_count_df.loc[6,"Total Count"] = player_count_df.loc[6,"Total Count"] + 1
    else:
        player_count_df.loc[7,"Total Count"] = player_count_df.loc[7,"Total Count"] + 1

total_player_count = unique_df["SN"].count()

# Calculage percentage of players in the age group
player_count_df["Percentage of Players"] = player_count_df["Total Count"].apply(lambda x: 100 * x / float(total_player_count))

# Changing format of percentage of players
player_count_df["Percentage of Players"] = player_count_df["Percentage of Players"].map("{:,.2f}%".format)

player_count_df

,Age Ranges,Total Count,Percentage of Players
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

